In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('combine_data_clean.csv')
df.head()

,id,title,userId,rating
0,862,Toy Story,1923,3.0
1,862,Toy Story,2103,5.0
2,862,Toy Story,5380,1.0
3,862,Toy Story,6177,4.0
4,862,Toy Story,6525,4.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1048575 non-null  int64  
 1   title   1048575 non-null  object 
 2   userId  1048575 non-null  int64  
 3   rating  1048575 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 32.0+ MB


In [4]:
df = df.drop(['userId'], axis = 1)

In [5]:
df1=df.groupby('id').mean().reset_index()
df2=df.groupby('id')['rating'].nunique().reset_index()
dfx= pd.merge(df1, df2, on='id', how='outer')
dfx= dfx.rename(columns={'rating_x': 'average', 'rating_y': 'count'})

vote_counts = dfx[dfx['count'].notnull()]['count'].astype('int')
vote_averages = dfx[dfx['average'].notnull()]['average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.9)
def weighted_rating(data):
    v = data['count'] 
    R = data['average']
    return (v / (v + m) * R) + (m / (m + v) * C)

dfx['weighted_rating'] = dfx.apply(weighted_rating, axis=1)


In [6]:
dfx = dfx.sort_values(by=['weighted_rating'], ascending=False)
dff =pd.merge(dfx, df, on='id')
dff = dff.drop_duplicates(subset=['id'])
dff.head(10)

,id,average,count,weighted_rating,title,rating
0,858,4.339811,10,3.541384,Sleepless in Seattle,5.0
57070,527,4.266531,10,3.504744,Once Were Warriors,4.0
124732,2019,4.255074,10,3.499016,Hard Target,5.0
138726,922,4.200820,10,3.471889,Dead Man,4.0
146656,1945,4.134570,10,3.438764,Nell,5.0
153069,260,4.132299,10,3.437629,The 39 Steps,4.0
230114,1939,4.084093,10,3.413525,Laura,4.0
232088,306,4.059447,10,3.401202,Beverly Hills Cop III,5.0
240440,213,4.030479,10,3.386718,North by Northwest,5.0
242425,909,4.024031,10,3.383494,Meet Me in St. Louis,4.0
